In [17]:
import json
import pandas as pd
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
import warnings

## Read from Kafka topic: assessment

In [9]:
raw_assessment = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:29092") \
  .option("subscribe","assessment") \
  .option("startingOffsets", "earliest") \
  .option("endingOffsets", "latest") \
  .load() 

In [10]:
raw_assessment.cache()

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [11]:
raw_assessment.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [12]:
assessment = raw_assement.select(raw_assement.value.cast('string'))

In [24]:
assessment.write.parquet("/tmp/players")

In [38]:
assessment.show(4)

+--------------------+
|               value|
+--------------------+
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
|{"keen_timestamp"...|
+--------------------+
only showing top 4 rows



In [14]:
assessment.collect()[0]

Row(value='{"keen_timestamp":"1516717442.735266","max_attempts":"1.0","started_at":"2018-01-23T14:23:19.082Z","base_exam_id":"37f0a30a-7464-11e6-aa92-a8667f27e5dc","user_exam_id":"6d4089e4-bde5-4a22-b65f-18bce9ab79c8","sequences":{"questions":[{"user_incomplete":true,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:24.670Z","id":"49c574b4-5c82-4ffd-9bd1-c3358faf850d","submitted":1,"correct":true},{"checked":true,"at":"2018-01-23T14:23:25.914Z","id":"f2528210-35c3-4320-acf3-9056567ea19f","submitted":1,"correct":true},{"checked":false,"correct":true,"id":"d1bf026f-554f-4543-bdd2-54dcf105b826"}],"user_submitted":true,"id":"7a2ed6d3-f492-49b3-b8aa-d080a8aad986","user_result":"missed_some"},{"user_incomplete":false,"user_correct":false,"options":[{"checked":true,"at":"2018-01-23T14:23:30.116Z","id":"a35d0e80-8c49-415d-b8cb-c21a02627e2b","submitted":1},{"checked":false,"correct":true,"id":"bccd6e2e-2cef-4c72-8bfa-317db0ac48bb"},{"checked":true,"at":"2018-01-23T14:23:41.

## Spark Infered Schema

In [18]:
extracted_assessment = assessment.rdd.map(lambda x: json.loads(x.value)).toDF()

/spark-2.2.0-bin-hadoop2.6/python/pyspark/sql/session.py:351: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [20]:
extracted_assessment.show()

+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|        base_exam_id|certification|           exam_name|   keen_created_at|             keen_id|    keen_timestamp|max_attempts|           sequences|          started_at|        user_exam_id|
+--------------------+-------------+--------------------+------------------+--------------------+------------------+------------+--------------------+--------------------+--------------------+
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717442.735266|5a6745820eb8ab000...| 1516717442.735266|         1.0|Map(questions -> ...|2018-01-23T14:23:...|6d4089e4-bde5-4a2...|
|37f0a30a-7464-11e...|        false|Normal Forms and ...| 1516717377.639827|5a674541ab6b0a000...| 1516717377.639827|         1.0|Map(questions -> ...|2018-01-23T14:21:...|2fec1534-b41f-441...|
|4beeac16-bb83-4d5...|        false

In [22]:
extracted_assessment.printSchema()

root
 |-- base_exam_id: string (nullable = true)
 |-- certification: string (nullable = true)
 |-- exam_name: string (nullable = true)
 |-- keen_created_at: string (nullable = true)
 |-- keen_id: string (nullable = true)
 |-- keen_timestamp: string (nullable = true)
 |-- max_attempts: string (nullable = true)
 |-- sequences: map (nullable = true)
 |    |-- key: string
 |    |-- value: array (valueContainsNull = true)
 |    |    |-- element: map (containsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: boolean (valueContainsNull = true)
 |-- started_at: string (nullable = true)
 |-- user_exam_id: string (nullable = true)



In [17]:
extracted_assement.registerTempTable('assement')

## Final Schema

In [24]:
spark.sql("select exam_name, certification, keen_id from assement").show()

+--------------------+-------------+--------------------+
|           exam_name|certification|             keen_id|
+--------------------+-------------+--------------------+
|Normal Forms and ...|        false|5a6745820eb8ab000...|
|Normal Forms and ...|        false|5a674541ab6b0a000...|
|The Principles of...|        false|5a67999d3ed3e3000...|
|The Principles of...|        false|5a6799694fc7c7000...|
|Introduction to B...|        false|5a6791e824fccd000...|
|        Learning Git|        false|5a67a0b6852c2a000...|
|Git Fundamentals ...|        false|5a67b627cc80e6000...|
|Introduction to P...|        false|5a67ac8cb0a5f4000...|
|Intermediate Pyth...|        false|5a67a9ba060087000...|
|Introduction to P...|        false|5a67ac54411aed000...|
|A Practical Intro...|        false|5a67ad9b2ff312000...|
|Git Fundamentals ...|        false|5a67b610baff90000...|
|Introduction to M...|        false|5a67ac9837b82b000...|
|   Python Epiphanies|        false|5a67aaa4f21cc2000...|
|Introduction 

In [42]:
final_schema = StructType(
    [StructField('user_exam_id', StringType(), True),
     StructField('base_exam_id', StringType(), True),
     StructField('keen_id', StringType(), True),
     StructField('exam_name', StringType(), True),
     StructField('certification', StringType(), True),
     StructField('keen_timestamp', StringType(), True),
     StructField('sequences',StructType(
         [StructField('counts',StructType([
             StructField('incomplete',IntegerType(), True),
             StructField('submitted',IntegerType(), True),
             StructField('incorrect',IntegerType(), True),
             StructField('all_correct',StringType(), True),
             StructField('correct',IntegerType(), True),
             StructField('total',IntegerType(), True),
             StructField('unanswered',IntegerType(), True)
         ]))]))
    ])


In [43]:
select_extracted_assessment = assessment.rdd.map(lambda x: json.loads(x.value)).toDF(schema=final_schema)

In [44]:
select_extracted_assessment.registerTempTable('select_assessment')

In [45]:
select_extracted_assessment.printSchema()

root
 |-- user_exam_id: string (nullable = true)
 |-- base_exam_id: string (nullable = true)
 |-- keen_id: string (nullable = true)
 |-- exam_name: string (nullable = true)
 |-- certification: string (nullable = true)
 |-- keen_timestamp: string (nullable = true)
 |-- sequences: struct (nullable = true)
 |    |-- counts: struct (nullable = true)
 |    |    |-- incomplete: integer (nullable = true)
 |    |    |-- submitted: integer (nullable = true)
 |    |    |-- incorrect: integer (nullable = true)
 |    |    |-- all_correct: string (nullable = true)
 |    |    |-- correct: integer (nullable = true)
 |    |    |-- total: integer (nullable = true)
 |    |    |-- unanswered: integer (nullable = true)



In [47]:
select_extracted_assessment.show(4)

+--------------------+--------------------+--------------------+--------------------+-------------+------------------+--------------------+
|        user_exam_id|        base_exam_id|             keen_id|           exam_name|certification|    keen_timestamp|           sequences|
+--------------------+--------------------+--------------------+--------------------+-------------+------------------+--------------------+
|6d4089e4-bde5-4a2...|37f0a30a-7464-11e...|5a6745820eb8ab000...|Normal Forms and ...|        false| 1516717442.735266|[[1,4,1,false,2,4...|
|2fec1534-b41f-441...|37f0a30a-7464-11e...|5a674541ab6b0a000...|Normal Forms and ...|        false| 1516717377.639827|[[2,4,1,false,1,4...|
|8edbc8a8-4d26-429...|4beeac16-bb83-4d5...|5a67999d3ed3e3000...|The Principles of...|        false| 1516738973.653394|[[0,4,1,false,3,4...|
|c0ee680e-8892-4e6...|4beeac16-bb83-4d5...|5a6799694fc7c7000...|The Principles of...|        false|1516738921.1137421|[[2,4,0,false,2,4...|
+-------------------

## How many assesstments are in the dataset?

In [55]:
spark.sql("select count(keen_id) from select_assessment").show()

+--------------+
|count(keen_id)|
+--------------+
|          3280|
+--------------+



In [51]:
spark.sql("select count(distinct keen_id) from select_assessment").show()

+-----------------------+
|count(DISTINCT keen_id)|
+-----------------------+
|                   3242|
+-----------------------+



In [53]:
spark.sql("select count(distinct base_exam_id) from select_assessment").show()

+----------------------------+
|count(DISTINCT base_exam_id)|
+----------------------------+
|                         107|
+----------------------------+



In [54]:
spark.sql("select count(distinct user_exam_id) from select_assessment").show()

+----------------------------+
|count(DISTINCT user_exam_id)|
+----------------------------+
|                        3242|
+----------------------------+



## How many people took Learning Git?


In [68]:
spark.sql("select count(distinct user_exam_id) as People_No from select_assessment WHERE exam_name = 'Learning Git'").show()

+---------+
|People_No|
+---------+
|      390|
+---------+



## What is the least common course taken? And the most common?

In [63]:
spark.sql("select count(user_exam_id) as popularity, exam_name from select_assessment group by exam_name order by popularity").show(20,False)


+----------+---------------------------------------------------+
|popularity|exam_name                                          |
+----------+---------------------------------------------------+
|1         |Learning to Visualize Data with D3.js              |
|1         |Native Web Apps for Android                        |
|1         |Nulls, Three-valued Logic and Missing Information  |
|1         |Operating Red Hat Enterprise Linux Servers         |
|2         |The Closed World Assumption                        |
|2         |Client-Side Data Storage for Web Developers        |
|2         |Arduino Prototyping Techniques                     |
|2         |Understanding the Grails 3 Domain Model            |
|2         |Hibernate and JPA Fundamentals                     |
|2         |What's New in JavaScript                           |
|2         |Learning Spring Programming                        |
|3         |Mastering Web Views                                |
|3         |Using Web Com

In [61]:
spark.sql("select count(user_exam_id) as popularity, exam_name from select_assessment group by exam_name order by popularity DESC").show(5, False)

+----------+-------------------------------+
|popularity|exam_name                      |
+----------+-------------------------------+
|394       |Learning Git                   |
|162       |Introduction to Python         |
|158       |Introduction to Java 8         |
|158       |Intermediate Python Programming|
|128       |Learning to Program with R     |
+----------+-------------------------------+
only showing top 5 rows

